The following additional libraries are needed to run this
notebook. Note that running on Colab is experimental, please report a Github
issue if you have any problem.

In [1]:
!pip install -q "numpy==2.0.2" "pandas==2.2.2" "matplotlib==3.10.0" "matplotlib-inline>=0.1.7" "jedi>=0.16"
!pip install -q --no-deps d2l==1.0.3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.7/111.7 kB 2.0 MB/s eta 0:00:00


# Custom Layers

One factor behind deep learning's success
is the availability of a wide range of layers
that can be composed in creative ways
to design architectures suitable
for a wide variety of tasks.
For instance, researchers have invented layers
specifically for handling images, text,
looping over sequential data,
and
performing dynamic programming.
Sooner or later, you will need
a layer that does not exist yet in the deep learning framework.
In these cases, you must build a custom layer.
In this section, we show you how.


In [2]:
import torch
from torch import nn
from torch.nn import functional as F
from d2l import torch as d2l

## (**Layers without Parameters**)

To start, we construct a custom layer
that does not have any parameters of its own.
This should look familiar if you recall our
introduction to modules in :numref:`sec_model_construction`.
The following `CenteredLayer` class simply
subtracts the mean from its input.
To build it, we simply need to inherit
from the base layer class and implement the forward propagation function.


In [3]:
class CenteredLayer(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, X):
        return X - X.mean()

In [5]:
#
class work_CenteredLayer(nn.Module):
  def __init__(self):
    super().__init__()

  def forward(self, X):
    return X - X.mean()

Let's verify that our layer works as intended by feeding some data through it.


In [6]:
layer = CenteredLayer()
layer(torch.tensor([1.0, 2, 3, 4, 5]))

tensor([-2., -1.,  0.,  1.,  2.])

In [14]:
#
layer_1 = work_CenteredLayer()
layer_1(torch.tensor([1.0, 2, 3, 4, 5]))

tensor([-2., -1.,  0.,  1.,  2.])

We can now [**incorporate our layer as a component
in constructing more complex models.**]


In [15]:
net = nn.Sequential(nn.LazyLinear(128), CenteredLayer())

In [16]:
#
work = nn.Sequential(nn.LazyLinear(128), work_CenteredLayer())

As an extra sanity check, we can send random data
through the network and check that the mean is in fact 0.
Because we are dealing with floating point numbers,
we may still see a very small nonzero number
due to quantization.


In [27]:
Y = net(torch.rand(4, 8))
Y.mean()

tensor(5.5879e-09, grad_fn=<MeanBackward0>)

In [28]:
#
Y1 = work(torch.rand(100, 100))
Y1.mean()

tensor(-3.5763e-09, grad_fn=<MeanBackward0>)

## [**Layers with Parameters**]

Now that we know how to define simple layers,
let's move on to defining layers with parameters
that can be adjusted through training.
We can use built-in functions to create parameters, which
provide some basic housekeeping functionality.
In particular, they govern access, initialization,
sharing, saving, and loading model parameters.
This way, among other benefits, we will not need to write
custom serialization routines for every custom layer.

Now let's implement our own version of the  fully connected layer.
Recall that this layer requires two parameters,
one to represent the weight and the other for the bias.
In this implementation, we bake in the ReLU activation as a default.
This layer requires two input arguments: `in_units` and `units`, which
denote the number of inputs and outputs, respectively.


In [29]:
class MyLinear(nn.Module):
    def __init__(self, in_units, units):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(in_units, units))
        self.bias = nn.Parameter(torch.randn(units,))

    def forward(self, X):
        linear = torch.matmul(X, self.weight.data) + self.bias.data
        return F.relu(linear)

In [52]:
#
class work_Linear(nn.Module):
  def __init__(self, ins, outs):
    super().__init__()
    self.weight = nn.Parameter(torch.randn(ins, outs))
    self.bias = nn.Parameter(torch.randn(outs,))

  def forward(self, X):
    linear = torch.matmul(X, self.weight.data) + self.bias.data
    return F.relu(linear)

Next, we instantiate the `MyLinear` class
and access its model parameters.


In [31]:
linear = MyLinear(5, 3)
linear.weight

Parameter containing:
tensor([[-0.1196, -0.2002,  0.4388],
        [-0.0795, -0.9848, -0.4441],
        [ 0.9998,  0.7864,  0.2504],
        [-0.3347, -1.0106,  1.4494],
        [-0.5552,  1.0576,  0.5126]], requires_grad=True)

In [56]:
#
lines = work_Linear(4, 3)
lines.weight, lines.bias

(Parameter containing:
 tensor([[-0.0732, -1.0562, -0.2309],
         [-0.5994, -1.6343, -0.0648],
         [ 0.5172,  0.0864, -0.5322],
         [ 0.2474,  0.2815,  0.7592]], requires_grad=True),
 Parameter containing:
 tensor([ 0.3811, -0.3631, -0.2356], requires_grad=True))

We can [**directly carry out forward propagation calculations using custom layers.**]


In [50]:
linear(torch.rand(2, 5))

tensor([[0.0000, 0.2124, 0.8337],
        [0.0000, 0.5260, 1.7051]])

In [59]:
#
lines(torch.rand(3, 4))

tensor([[0.7667, 0.0000, 0.0000],
        [0.4917, 0.0000, 0.0000],
        [0.1640, 0.0000, 0.1210]])

We can also (**construct models using custom layers.**)
Once we have that we can use it just like the built-in fully connected layer.


In [71]:
net = nn.Sequential(MyLinear(64, 8), MyLinear(8, 1))
net(torch.rand(2, 64))

tensor([[6.7751],
        [9.9968]])

In [69]:
work = nn.Sequential(work_Linear(128, 32), work_Linear(32, 1))
work(torch.randn(2, 128))

tensor([[23.3893],
        [ 0.0000]])

In [79]:
# Exercise 1
class tensor_reduction(nn.Module):
  def __init__(self):
    super().__init__()
    self.line = nn.LazyLinear(192)

  def forward(self, X):
    return torch.sum(self.line(X))

ex1 = tensor_reduction()
ex1(torch.randn(2, 128))

tensor(-8.2640, grad_fn=<SumBackward0>)

## Summary

We can design custom layers via the basic layer class. This allows us to define flexible new layers that behave differently from any existing layers in the library.
Once defined, custom layers can be invoked in arbitrary contexts and architectures.
Layers can have local parameters, which can be created through built-in functions.


## Exercises

1. Design a layer that takes an input and computes a tensor reduction,
   i.e., it returns $y_k = \sum_{i, j} W_{ijk} x_i x_j$.
1. Design a layer that returns the leading half of the Fourier coefficients of the data.


1. Done

2. No thanks

[Discussions](https://discuss.d2l.ai/t/59)
